<a href="https://colab.research.google.com/github/biggojo/Assignments-DL/blob/main/Exercise2a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### This submission is for:
- Abraham Gassama (2285843)
- Mikail Eroglu (2281738)
- Zhuo Le Lee (2317240)

# Exercise 2A - Transformers

In this exercise, you'll implement a basic encoder-only Transformer architecture with PyTorch. We will start with building the basic building blocks and then integrate them into a fully-fleged Transformer model. We train the model to solve a POS-Tagging problem (more on that later). In the previous exercise, you implemented your work in numpy. Now, we will switch to PyTorch, which will track the gradients for us and allows us to focus more on the network itself.

You can receive up to three points for your implementation of Exercise 2A. Together with Exercise 1, you can get up to six bonus points for the exam.

**Important Notice**: Throughout the notebook, basic structures are provided such as functions and classes without bodies or partial bodies, and variables that you need to assign to. **Don't change the names of functions, variables, and classes - and make sure that you are using them!** You're allowed to introduce helper variables and functions. Occasionally, we use **type annotations** that you should follow. They are not enforced by Python. Whenenver you see an ellipsis `...` you're supposed to insert code.

In [15]:
!pip install torchtext torchdata torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [16]:
import torch
from torch import nn, Tensor

Let's actually start with a few basic functions that we will need throughout the exercise, namely **Softmax** and **ReLu**.

$\text{Softmax}(x_{i}) = \frac{\exp(x_i)}{\sum_j \exp(x_j)}$

$\text{ReLU}(x) = \max(0, x)$

In [17]:
def softmax(input: Tensor) -> Tensor:
    
    # Clone the input tensor and save as output tensor to be overwritten 
    output_tensor = torch.clone(input)
    
    # Note that this implementation assumes that the input tensor is a one directional array 
    # For a multidimensional input, the dimension along which the input elements are to be normalized should be given as an input parameter
    exp_input = torch.exp(input)
    sum_exp = torch.sum(exp_input)
    output_tensor = exp_input / sum_exp
    
    return output_tensor

def relu(input: Tensor) -> Tensor:
    return torch.maximum(input, torch.zeros(input.size()))
    


In [18]:
# Testing out certain pytorch functions
'''
sampleTensor = Tensor([10,8,2,3,5,8,10])
print(sampleTensor[0])                                                            # Tensor can be indexed and sliced exactly like arrays
print(sampleTensor.shape[0])

sampleTensorB = torch.clone(sampleTensor)                                         # Replicating tensors -> Memory inefficient, room for improvement
print(sampleTensorB)

TwoD_tensor = Tensor([[1,2,3],[4,5,6]])
sum_over_row = torch.sum(TwoD_tensor, dim=0)
print(sum_over_row)

exp_sampleTensor = torch.exp(sampleTensor)
print(exp_sampleTensor)

two_mul = Tensor([2,4,6,8,10,12,14,16,18])
one_mul = two_mul / 2
print(one_mul)

print(TwoD_tensor.size())
ones = torch.ones(TwoD_tensor.size())
print(ones)

'''

'\nsampleTensor = Tensor([10,8,2,3,5,8,10])\nprint(sampleTensor[0])                                                            # Tensor can be indexed and sliced exactly like arrays\nprint(sampleTensor.shape[0])\n\nsampleTensorB = torch.clone(sampleTensor)                                         # Replicating tensors -> Memory inefficient, room for improvement\nprint(sampleTensorB)\n\nTwoD_tensor = Tensor([[1,2,3],[4,5,6]])\nsum_over_row = torch.sum(TwoD_tensor, dim=0)\nprint(sum_over_row)\n\nexp_sampleTensor = torch.exp(sampleTensor)\nprint(exp_sampleTensor)\n\ntwo_mul = Tensor([2,4,6,8,10,12,14,16,18])\none_mul = two_mul / 2\nprint(one_mul)\n\nprint(TwoD_tensor.size())\nones = torch.ones(TwoD_tensor.size())\nprint(ones)\n\n'

In [19]:
# Sanity checks for the ReLU and softmax function

# Create a tensor of random values with 15 columns and 1 row to match the assumption made
sample_input = torch.rand(15)

# Note: The output tensors are rounded to 4 decimals places to allow for easier comparison
# Output from our implementation
SM_out1 = torch.round(softmax(sample_input), decimals=4)
ReLU_out1 = torch.round(relu(sample_input), decimals=4)

# Output from PyTorch Implementation
SM_pytorch = nn.Softmax(dim=0)
ReLU_pytorch = nn.ReLU()
SM_out2 = torch.round(SM_pytorch(sample_input), decimals=4)
ReLU_out2 = torch.round(ReLU_pytorch(sample_input), decimals=4)


softmax_compare = torch.eq(SM_out1, SM_out2)
relu_compare = torch.eq(ReLU_out1, ReLU_out2)


print(softmax_compare)
print(relu_compare)

# Both functions passed the sanity checks 

tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True])
tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True])


## Transformer Block

A typical transformer block consists of the following 
- Multi-Head Attention
- Layer Normalization
- Linear Layer
- Residual Connections

<center><img src="https://i.imgur.com/ZKgcoe4.png" alt="transformer block visualization" width="200">

In the next few subsections, we will build these basic building blocks.

### Multi-Head Attention

Multi-Head Attention concatenates the outputs of several so called **attention heads**.

$\textrm{MHA}(Q,K,V) = \textrm{Concat}(H_1,...,H_h)$

<center><img src="https://www.tensorflow.org/images/tutorials/transformer/multi_head_attention.png" width=300>

One attention head consists of linear projections for each of $Q, K$ and $V$ and an attention mechanism called **Scaled Dot-Product Attention**. The attention mechanism scales down the dot products by $\sqrt{d_k}$.

$\textrm{Attention}(Q,K,V)=\textrm{softmax}(\frac{QK^T}{\sqrt{d_k}})V$



If we assume that $q$ and $v$ are $d_k$-dimensional vectors and its components are independent random variables with mean $0$ and a variance of $d_k$, then their dot product has a mean of $0$ and variance of $d_k$. It is preferred to have a variance of $1$ and that's why we scale them down by $\sqrt{d_k}$.

The dot product $q \cdot v$ resembles a measure of similarity.


<center><img src="https://www.tensorflow.org/images/tutorials/transformer/scaled_attention.png" width="350">

Let's start implementing these components. Note that our classes inherit from PyTorch's `nn.Module`. These modules allow us to hold our parameters and easily move them to the GPU (with `.to(...)`). It also let's us define the computation that is performed at every call, in the `forward()` method. For example, when we have an `Attention` module, initialize it like `attention = Attention(...)`, we are able to call it with `attention(Q, K, V)` (it'll execute the `forward` function in an optimized way).

### Assumptions: 


1.   Q, K and V are **batches** of matrices, each with shape *(batch_size, seq_length, num_features)*. 
2.   The attention here is self-attention, this means that the sequence length of Q and K are the same. 
3.   The number of hidden dimensions for all Q, K and V are the same.
4.   For added simplicity, we first assume that the batch size is one. We then introduce batched attention matrix after the simple attention has passed all of the sanity checks

### Operations in Attention and their outputs: 



1.   Multiplying the query (Q) and key (K) arrays in a linear matrix multiplication. This is the attention of this layer and determines how important each element in the key sequences is with regard to the query sequence. 

> Output 1: *(batch_size, seq_length, seq_length)*

2.   Scaling the attention to have variance of 1, regardless of the size of seq_length

> Output 2: *(batch_size, seq_length, seq_length)*

3.   Normalizing the attention array across the keys dimension with softmax, so that all the attention weights sum to one. (Because we can't pay more than 100% attention!)

> Output 3: *(batch_size, seq_length, seq_length)*

4.   Multiplying the attention matrix with the value (V) array using matrix multiplication. 

> Output 4: *(batch_size, seq_length, num_features)*









In [20]:
# Implementation of a single attention head

class Attention(nn.Module):
    def __init__(self, hidden_n: int):
        """
        hidden_n : Number of hidden dimensions
        It is here assumed that the number of hidden dimensions corresponds
        to the number of features in each input timestep.
        """
        super().__init__()
        self.hidden_n = hidden_n
        
        """
        nn.Linear intializes and defines a transformation matrix, where all but 
        the last dimension of the output is different than the input 
        """
        self.Q_linear = nn.Linear(hidden_n, hidden_n)
        self.V_linear = nn.Linear(hidden_n, hidden_n)
        self.K_linear = nn.Linear(hidden_n, hidden_n)


    def forward(self, Q: Tensor, K: Tensor, V: Tensor, mask=None) -> Tensor:
        """
        Note 1: If it was a batched attention, then all of the following matmul 
        has to be changed to bmm (batched matrix multiplication, supported by
        pytorch)

        Note 2: The above softmax function would not work here, if the number of 
        time steps (i.e. sequence length) and the number of features are both
        more than one. The above softmax function would have to be revised to 
        include the number of dimensions.

        Note 3: The dimensions of the transpose has to be double-checked against
        the size of the inputs to the attention layer later, when the inputs
        are given.

        Note 4: A naive implementation of the masking operation is given here, 
        should the a mask be defined as an input. This might have to be revised
        in later steps.
        """

        # This step can be made redundant later, when the output dimensions 
        # matches that of expected.
        Q_arr = self.Q_linear(Q)
        K_arr = self.K_linear(K)
        V_arr = self.V_linear(V)

        scores = torch.matmul(Q, K.transpose(-1,1)) / (self.hidden_n ** 0.5)
        
        if mask is not None:
          mask = mask.unsqueeze(1)
          scores = scores.masked_fill(mask == 0, -1e9)
          scores = softmax(scores)
        
        scores = softmax(scores)

        output = torch.matmul(scores, V)
        return output 


In [21]:
class MultiHeadAttention(nn.Module):
    def __init__(self, hidden_n: int, h: int = 2):
        """
        hidden_n: hidden dimension
        h: number of heads
        """
        super().__init__()
        self.hidden_n = hidden_n
        self.h = h
        # self.d_k = hidden_n // h

        """
        Intialize an attention head for each element in h
        """
        self.heads = nn.ModuleList(
            [Attention(hidden_n) for x in range(h)]
        )
        
        """
        After concatenating, the output size would be (batch_size, seq_length, num_features, h)
        This can be reverted back to the original output size via a linear layer

        Note 1: There are other implementation variations regarding this step.
        E.g. we could split the input into n heads, so that each will have the dimensions
        (batch_size, seq_length, num_features, num_features / n), with the last dimension denoting
        which head each input is subjected to. Each input is then passed through the according
        attention head and then concatenated. 

        This variation was not chosen for simplicity, e.g. self.d_k has to be a factor of 
        self.hidden_n
        """
        self.out = nn.Linear(h * hidden_n, hidden_n)

   
    def forward(self, Q: Tensor, K: Tensor, V: Tensor, mask=None):
        return self.out(
            torch.cat([h(Q,K,V) for h in self.heads], dim=-1)
        )

### Layer Normalization

From the lecture, remember layer normalization where the values are normalized across the feature dimension, independently for each sample in the batch. For that, first calculate mean and standard-deviation across the feature dimension and then scale them appropriately such that the mean is 0 and the standard deviation is 1. Introduce **two sets of learnable parameters**, one for shifting the mean (addition) and one for scaling the variance (multiplication) the normalized features (i.e., two parameters for each feature). Tip: Use `nn.Parameter` for that.

$y_{\textrm{norm}}=\frac{x-\mu}{\sqrt{\sigma+\epsilon}}$

$y=y_{\textrm{norm}}\cdot\beta+\alpha$

<center>
<img src="https://i.stack.imgur.com/E3104.png" alt="visualization of layer norm vs. batch norm" width="420">

In [22]:
class LayerNorm(nn.Module):
    def __init__(self, norm_shape):
        """
        See also: [Layer Normalization](https://arxiv.org/pdf/1607.06450.pdf)
        
        :param norm_shape: The dimension of the layer to be normalized, i.e. the 
        shape of the input tensor or the last dimension of the input tensor.

        Note 1: It is assummed here that norm_shape is the 
        """

        super().__init__()
        self.norm_shape = norm_shape

        """
        :param alpha: Add a scale parameter if it is True.
        :param beta: Add an offset parameter if it is True.
        :param epsilon: Epsilon for calculating variance.

        The initialized value of alpha and beta should be the same as the dimension
        of the layer to be normalized, assuming that the multiplication between
        beta and y_norm is element-wise.

        Note 1: It is often said that the parameters initialized with nn.Parameter()
        often have dimunitive values like 1.4013e-45. Other alternatives such as
        nn.Linear() might be worth considering, since it does initial processing to the 
        input tensor such as uniformization. 

        Note 2: Here, the weights are intialized to be random variables form a 
        uniform distribution on the interval [0,1)
        
        Note 3: The paramters can also be intialized with torch.Tensor(*norm_shape)
        or torch.empty(norm_shape)
        """
        self.alpha = torch.nn.Parameter(torch.rand(norm_shape))
        self.beta = torch.nn.Parameter(torch.rand(norm_shape))
        self.epsilon = 1e-10

    def forward(self, x):
        """
        Note 1: It is assumed that the feature dimension is the last dimension, which 
        matches our assumption from above.
        """
        mean = x.mean(dim=-1, keepdim=True)
        var = ((x - mean) ** 2).mean(dim=-1, keepdim=True)
        std = (var + self.epsilon).sqrt()
        y_norm = (x - mean) / std

        y = (y_norm * self.beta) + self.alpha
        return y

### Transformer Block

Here, we bring all ingredients together into a single module. Don't forget to add the residual connections.

In [27]:
class TransformerBlock(nn.Module):
    def __init__(self, hidden_n: int, h: int = 2):
        """
        hidden_n: hidden dimension
        h: number of heads
        """
        super().__init__()
        self.hidden_n = hidden_n
        self.h = h
        self.attention = MultiHeadAttention(hidden_n, h)
        self.norm1 = LayerNorm(hidden_n)
        self.norm2 = LayerNorm(hidden_n)

        # This following variable can be eventually defined via input 
        ff_dim = 2048
        self.feed_forward = nn.Sequential(
            nn.Linear(hidden_n, ff_dim*hidden_n),
            nn.ReLU(),
            nn.Linear(ff_dim*hidden_n, hidden_n)
        )

    # Assuming that the query, key and value arrays are the same and given as input
    def forward(self, x: Tensor):
        """
        Alternatively, the inputs could be normalized first before using them as
        input for the attention layer, while the residual connections are not normalised

        x_norm1 = self.norm1(x)
        x_out1 = x + self.attention(x_norm1, x_norm1, x_norm1)
        x_norm2 = self.norm_2(x_out1)
        x_out2 = x_out1 + self.feed_forward(x_norm2)

        return x_out2

        """
        
        x_out1 = x + self.attention(x, x, x)
        x_attn = self.norm1(x)

        x_out2 = x_attn + self.feed_forward(x_attn)
        x_ff = self.norm2(x_out2)
        return x_ff


## A Simple Transformer Architecture

Let's stack our transformer blocks and add an embedding layer for a simple transformer architecture. You are allowed to use `nn.Embedding` here.

In [ ]:
class Transformer(nn.Module):
    def __init__(self, emb_n: int, hidden_n: int, n:int =3, h:int =2):
        """
        emb_n: number of token embeddings
        hidden_n: hidden dimension
        n: number of layers
        h: number of heads per layer
        """
        super().__init__()
        ...

    def forward(self):
        ...

## POS-Tagging

Part-Of-Speech-Tagging (**POS-Tagging**) is a **sequence labeling problem** where we categorize words in a text in correspondence with a particular part of speech (e.g., "noun" or "adjective"). A few examples and classes are shown in the following table:

|  POS Tag  |  Description  |  Examples  |
|-----------|------------|------------|
|  NN | Noun (singular, common) | mass, wind, ...  |
|  NNP | Noun (singular, proper) | Obama, Liverpool, ...  |
| CD  | Numeral (cardinal)  | 1890, 0.5, ...  |
|  DT | Determiner  | all, any, ... |
| JJ | Adjective (ordinal) | oiled, third, ... |
... many more

### CoNLL2000 Dataset

Let's load our dataset which is the **CoNLL2000 dataset** and look at an example.

In [ ]:
from torch.utils.data import Dataset, DataLoader
from torchtext.datasets import CoNLL2000Chunking
import pandas as pd

train_df = pd.DataFrame(CoNLL2000Chunking()[0], columns=['words', 'pos_tags', 'chunk'])
test_df = pd.DataFrame(CoNLL2000Chunking()[1], columns=['words', 'pos_tags', 'chunk'])

train_src, train_tgt = train_df['words'].tolist(), train_df['pos_tags'].tolist()
test_src, test_tgt = test_df['words'].tolist(), test_df['pos_tags'].tolist()

print(train_src[0])
print(train_tgt[0])

['Confidence', 'in', 'the', 'pound', 'is', 'widely', 'expected', 'to', 'take', 'another', 'sharp', 'dive', 'if', 'trade', 'figures', 'for', 'September', ',', 'due', 'for', 'release', 'tomorrow', ',', 'fail', 'to', 'show', 'a', 'substantial', 'improvement', 'from', 'July', 'and', 'August', "'s", 'near-record', 'deficits', '.']
['NN', 'IN', 'DT', 'NN', 'VBZ', 'RB', 'VBN', 'TO', 'VB', 'DT', 'JJ', 'NN', 'IN', 'NN', 'NNS', 'IN', 'NNP', ',', 'JJ', 'IN', 'NN', 'NN', ',', 'VB', 'TO', 'VB', 'DT', 'JJ', 'NN', 'IN', 'NNP', 'CC', 'NNP', 'POS', 'JJ', 'NNS', '.']


First, we need to create a vocabulary. Our dataset is already tokenized. However, we need to assign ids to them in order to input them to the embedding layer. We also need the number of embeddings (`num_embeddings`) for the size of our lookup table of `nn.Embedding`.

Thus, we will iterate over all sentences replace them with ids and the mapping to our vocabulary. It'll be handy to have two different mappings, from id to token, as well as, from token to id. Note that we will add a special token `<unk>` with id `0` for words that are unknown (that are not in the training dataset but could possibly be in the test dataset).

In [ ]:
vocabulary_id2token : dict = {0: '<unk>'}
vocabulary_token2id : dict = {'<unk>': 0}

for sentence in train_src:
    ...

Let's do the same for our classes:

In [ ]:
classes_id2name : dict = {}
classes_name2id : dict = {}

for sentence in *[train_src, test_src]:
    ...

Now, let's use PyTorch's `Dataset` and `DataLoader` for help us batching our data. Let's also replace tokens and classes with our ids. For that, complete `get_token_ids` and `get_class_ids`.

In [ ]:
def get_token_ids(src: list[str]) -> list[int]:
    ...

def get_class_ids(tgt: list[str]) -> list[int]:
    ...

class ConllDataset(Dataset):
  def __init__(self, src, tgt):
        self.src = src
        self.tgt = tgt

  def __len__(self):
        return len(self.src)

  def __getitem__(self, index):
        src = self.src[index]
        tgt = self.tgt[index]
        
        return {
            'src': get_token_ids(src),
            'tgt': get_class_ids(tgt),
        }

train_dataset = ConllDataset(train_src, train_tgt)
test_dataset = ConllDataset(test_src, test_tgt)

We will use a **batch size of 32**.

In [ ]:
BATCH_SIZE = 32

However, since our examples are of different length, we need to pad shorter examples to the length of the example with the maximum length in our batch. So, let's define a special **padding token** in our vocabulary:

In [ ]:
padding_token = ...

vocabulary_id2token ...
vocabulary_token2id ...

The `collate_fn` is the function that actually receives a batch and needs to add the padding tokens, then returns `src` and `tgt` as `Tensor`s of size `[B, S]` where `B` is our batch size and `S` our maximum sequence length. This function should additionally return a `mask`, a `Tensor` with binary values to indicate whether the specific element is a padding token or not (0 if it's a padding token, 1 if not), such that we can ignore padding tokens in our attention mechanism and loss calculation. 

In [ ]:
def collate_fn(batch: list[dict]) -> dict[str, Tensor]:
    """
    batch: list of dictionaries with keys src and tgt (as defined in ConllDataset)
    """
    ...
    return {
        'src': src,
        'tgt': tgt,
        'mask': mask,
    }

With that, we can use PyTorch's `DataLoader` which will shuffle and batch our data automatically.

In [ ]:
train_data_loader = DataLoader(train_dataset, collate_fn=collate_fn, batch_size=BATCH_SIZE, shuffle=True)
test_data_loader = DataLoader(test_dataset, collate_fn=collate_fn, batch_size=BATCH_SIZE, shuffle=True)

### Architecture

Let's build a transformer model with three layers, three attention heads and an embedding dimension of 128. Also, let's not forget to add a classification head to our model.

In [ ]:
class CoNLL2000Transformer:
    def __init__(self, transformer, ...):
        super().__init__()
        self.transformer = transformer
        self.classification_layer = ...

    def forward(self):
        ...

model = CoNLL2000Transformer(Transformer(...), ...)

### Training

Initialize the **AdamW** optimizer from the `torch.optim` module and choose the most appropriate loss function for our task.

In [ ]:
optimizer = ....
criterion = ...

Build a basic training loop and train the network for three epochs.
- Use everything we've built to far, including `train_data_loader`, `model`, `optimizer` and `criterion`.
- At every 50th step print the average loss of the last 50 steps. 
- It is suggested to make a basic training procedure to work on the CPU first. Once it successfully runs on the CPU, you can switch to the GPU (click on change runtime and add an hardware accelerator if you use Colab) and run for the whole three epochs. Note: For this to work, you need to transfer the `model` and the input tensors to the GPU memory. This simply works by calling `.to(device)` on the model and tensors, where `device` and either be `cpu` or `cuda` (for the GPU).

In [ ]:
DEVICE = 'cpu' # later replace with 'cuda' for GPU
EPOCHS = 3

model = model.to(device)

for epoch in range(EPOCHS):
    ...

### Evaluation

Let's see what's the accuracy is of our model. Since we already implemented accuracy in the previous exercise, we'll now let you use the torchmetrics package.

In [ ]:
from torchmetrics import Accuracy

accuracy = Accuracy(average='micro')

Calculate the average accuracy of all examples in the test dataset.

In [ ]:
...

Let's also look at the accuracy **for each class separately**:

In [ ]:
...

## Positional Embeddings

The attention mechanism does not consider the position of the tokens which hurts its performance for many problems. We can solve this issue in several ways. We can either add a positional encoding (via trigonometric functions) or we can learn positional embeddings along the way, in a similar way as BERT does. Here, we will add learnable positional embeddings to our exisisting model with another embedding layer.

The longest sequence in our dataset has 78 tokens (you can trust us on that). So, let's set the number of embeddings for our positional embedding layer to that number. Again, you should use `nn.Embedding`.

Copy the inner parts of your `Transformer` class and add positional embeddings to it.

In [ ]:
class TransformerPos(nn.Module):
    def __init__(self, emb_n: int, pos_emb_n: int, hidden_n: int, n:int =3, h:int =2):
        """
        emb_n: number of token embeddings
        pos_emb_n: number of position embeddings
        hidden_n: hidden dimension
        n: number of layers
        h: number of heads per layer
        """
        super().__init__()
        self.positional_embeddings = ...
        ...

    def forward(self):
        ...

In [ ]:
model_pos = CoNLL2000Transformer(TransformerPos(...), ...)

### Training

Same procedure as before. Let's reinitialize our optimizer and our loss function and run the same training loop with our new model `model_pos`.

In [ ]:
optimizer = ....
criterion = ...

In [ ]:
...

### Evaluation

Now, let's check if our performance on the accuracy got improved.

In [ ]:
...

Again, let's also check each class. Which classes got improved the most by adding positional embeddings?

In [ ]:
...

The last question in this assignment doesn't require you to code anything. Instead, you're asked to point out possible issues with our current approach and name potential improvements. 
* ...
* ...
* ...

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=faa4af3b-d086-4f42-8b7d-d29c91b1d0f6' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>